These are some resources and readings that I've encountered that may be useful for students in mathematics and related fields. I've saved a random assortment of these over the years and so compiling this table is somewhat an exercise in cleaning up my own bookmarks! 


---


In [1]:
#| echo: false
from pathlib import Path
import yaml, html
from IPython.display import Markdown

# locate resources.yml (same strategy as before)
cwd = Path.cwd()
candidates = [cwd / "resources.yml", cwd / "data" / "resources.yml", cwd.parent / "resources.yml"]
datafile = next((p for p in candidates if p.exists()), candidates[0])

if not datafile.exists():
    raise FileNotFoundError("resources.yml not found. Searched: " + ", ".join(str(p) for p in candidates))

with open(datafile, encoding="utf-8") as fh:
    items = yaml.safe_load(fh) or []

# infer all keys across items
all_keys = set()
for it in items:
    all_keys.update(it.keys())

# prefer these columns (if present) and then append any remaining keys sorted
preferred_order = [
    "title", "organization", "audience", "stage", "type", "deadline",
    "location", "duration", "stipend", "tags", "notes", "apply_url", "url",
    "id", "last_updated"
]
# build ordered list
ordered = []
for k in preferred_order:
    if k in all_keys:
        ordered.append(k)
        all_keys.remove(k)
# append leftover keys alphabetically
ordered += sorted(all_keys)

# if title exists, prefer linking the title and drop raw url/apply_url columns
if "title" in ordered:
    ordered = [k for k in ordered if k not in ("url", "apply_url")]

# helper to stringify cell values
def render_value(v):
    if v is None:
        return ""
    if isinstance(v, list):
        # join lists; if list items are dicts, stringify
        parts = []
        for x in v:
            if isinstance(x, dict):
                parts.append(", ".join(f"{html.escape(str(a))}: {html.escape(str(b))}" for a,b in x.items()))
            else:
                parts.append(html.escape(str(x)))
        return ", ".join(parts)
    if isinstance(v, dict):
        return ", ".join(f"{html.escape(str(a))}: {html.escape(str(b))}" for a,b in v.items())
    return html.escape(str(v))

# build table HTML
rows = []
# include DataTables CSS + small styles (keep from previous chunk)
rows.append('''<link rel="stylesheet" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css"/>
<style>
  #resources-table { width: 100%; border-collapse: collapse; margin-top: 1rem; }
  #resources-table th, #resources-table td { padding: 0.5rem 0.75rem; vertical-align: top; }
  #resources-table th { background: rgba(0,0,0,0.04); font-weight: 600; text-align: left; }
  #resources-table td.title { max-width: 28rem; white-space: normal; }
  .resource-notes { color: #444; font-size: 0.95rem; }
  .small-muted { color: #666; font-size: 0.9rem; }
  a.resource-link { color: #0b66c3; text-decoration: none; }
  a.resource-link:hover { text-decoration: underline; }
</style>''')

rows.append('<div class="table-responsive">')
rows.append('<table id="resources-table" class="display" style="width:100%">')
rows.append('<thead>')
# header row (prettify column names)
header_cells = []
for k in ordered:
    label = k.replace("_", " ").title()
    header_cells.append(f"<th>{html.escape(label)}</th>")
rows.append("<tr>" + "".join(header_cells) + "</tr>")
rows.append('</thead>')
rows.append('<tbody>')

for it in items:
    rows.append("<tr>")
    for k in ordered:
        # special handling for title -> make it a link if apply_url/url present
        if k == "title":
            title_text = html.escape(it.get("title", "Untitled"))
            apply_url = it.get("apply_url") or ""
            url = it.get("url") or ""
            link = apply_url or url or ""
            if link:
                title_html = f'<a class="resource-link" href="{html.escape(link)}" target="_blank" rel="noopener">{title_text}</a>'
                # if both apply_url and url present and differ, show small helper
                extra = ""
                if it.get("apply_url") and it.get("url") and it.get("apply_url") != it.get("url"):
                    extra = f' <br><span class="small-muted"><a href="{html.escape(it.get("url"))}" target="_blank" rel="noopener">info</a> • <a href="{html.escape(it.get("apply_url"))}" target="_blank" rel="noopener">apply</a></span>'
                cell = title_html + extra
            else:
                cell = title_text
            rows.append(f"<td class='title'>{cell}</td>")
            continue

        value = it.get(k, "")
        cell_html = render_value(value)
        rows.append(f"<td>{cell_html}</td>")

    rows.append("</tr>")

rows.append('</tbody>')
rows.append('</table>')
rows.append('</div>')

# include jQuery, DataTables JS, and initialization script
rows.append('''
<script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
<script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
<script>
document.addEventListener("DOMContentLoaded", function(){
  $('#resources-table').DataTable({
    responsive: true,
    pageLength: 10,
    order: [[0,'asc'], [1, 'asc'], [2, 'asc']],
    orderMulti: true,
    columnDefs: [
      { targets: [], orderable: true } // you can tweak to disable specific columns if desired
    ],
    language: { search: "Filter:" }
  });
});
</script>
''')

res = "\n".join(rows)
Markdown(res)

<link rel="stylesheet" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css"/>
<style>
  #resources-table { width: 100%; border-collapse: collapse; margin-top: 1rem; }
  #resources-table th, #resources-table td { padding: 0.5rem 0.75rem; vertical-align: top; }
  #resources-table th { background: rgba(0,0,0,0.04); font-weight: 600; text-align: left; }
  #resources-table td.title { max-width: 28rem; white-space: normal; }
  .resource-notes { color: #444; font-size: 0.95rem; }
  .small-muted { color: #666; font-size: 0.9rem; }
  a.resource-link { color: #0b66c3; text-decoration: none; }
  a.resource-link:hover { text-decoration: underline; }
</style>
<div class="table-responsive">
<table id="resources-table" class="display" style="width:100%">
<thead>
<tr><th>Title</th><th>Audience</th><th>Type</th><th>Notes</th></tr>
</thead>
<tbody>
<tr>
<td class='title'><a class="resource-link" href="https://www.nsfgrfp.org/" target="_blank" rel="noopener">NSF Graduate Research Fellowship</a></td>
<td>Graduate</td>
<td>Funding</td>
<td>Major US graduate fellowship.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://math.stanford.edu/~vakil/potentialstudents.html" target="_blank" rel="noopener">For potential PhD students</a></td>
<td>Undergrad/Graduate</td>
<td>Advice</td>
<td>Now classic advice from Ravi Vakil on grad school.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://topology.pi-base.org" target="_blank" rel="noopener">Pi-Base</a></td>
<td>General</td>
<td>Math</td>
<td>A modern day companion to Counterexamples in Topology.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.lesswrong.com/posts/Sq75dZniBctPyTrWX/funding-is-all-you-need-getting-into-grad-school-by-hacking" target="_blank" rel="noopener">Funding is all you need...</a></td>
<td>Undergrad</td>
<td>Advice</td>
<td>Perspective on grad school applications from a non-traditional angle.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://gradsubgroups.org/" target="_blank" rel="noopener">SUBgroups</a></td>
<td>Graduate</td>
<td>Mentorship/Community</td>
<td>Peer groups for first year grad students.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.edgeforwomen.org/the-mary-beth-ruskai-research-fund-for-women/" target="_blank" rel="noopener">EDGE Research Fund for Women</a></td>
<td>General</td>
<td>Funding</td>
<td>Funding for women in the mathematical sciences. $5000 award.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.lathisms.org/scholarships" target="_blank" rel="noopener">Lathisms Scholarship</a></td>
<td>Undergrad/Graduate</td>
<td>Funding</td>
<td>Scholarship for Latinx/Hispanic students in the mathematical sciences. $500 award.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.slmath.org/msri-up" target="_blank" rel="noopener">MSRI-UP</a></td>
<td>Undergrad</td>
<td>Summer Program</td>
<td>Summer research program for undergrads at SLMath in Berkeley, CA.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://kconrad.math.uconn.edu/" target="_blank" rel="noopener">Keith Conrad&#x27;s Website</a></td>
<td>General</td>
<td>Math</td>
<td>Includes a large fantastic collection of handouts.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://virtualmath1.stanford.edu/~conrad/" target="_blank" rel="noopener">Brian Conrad&#x27;s Website</a></td>
<td>General</td>
<td>Math</td>
<td>Includes a large fantastic collection of handouts.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://geometrynyc.wixsite.com/polymathreu" target="_blank" rel="noopener">Polymath Jr REU</a></td>
<td>Undergrad</td>
<td>Summer Program</td>
<td>Online Summer math research program for undergrads.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://cst.temple.edu/department-mathematics/events/gta-philadelphia" target="_blank" rel="noopener">GTA Philadelphia</a></td>
<td>Graduate</td>
<td>Conference</td>
<td>Gradute student conference in Temple University, Philadelphia, PA.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://users.math.msu.edu/users/banelson/conferences/GOALS.html" target="_blank" rel="noopener">GOALS</a></td>
<td>Graduate</td>
<td>Summer Program</td>
<td>Graduate Summer school in operator algebras.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://categorified.net/TQFT2025/" target="_blank" rel="noopener">Atlantic TQFT Spring School</a></td>
<td>Graduate</td>
<td>Summer Program</td>
<td>Graduate spring school in TQFTs at the University of New Brunswick, Fredericton.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.salliemae.com/landing/bridging-the-dream-for-graduates/" target="_blank" rel="noopener">Bridging the Dream Scholarship</a></td>
<td>Graduate</td>
<td>Funding</td>
<td>Scholarship for graduate students. Around $10,000 award.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://mathmeetings.net/" target="_blank" rel="noopener">MathMeetings</a></td>
<td>General</td>
<td>Conference</td>
<td>Listing of upcoming math conferences worldwide.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://research.google/programs-and-events/phd-fellowship/" target="_blank" rel="noopener">Google PhD Fellowship</a></td>
<td>Graduate</td>
<td>Funding</td>
<td>Fellowship for PhD students in computational fields. 2 year award (in US).</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://gradpost.ucsb.edu/ips-program" target="_blank" rel="noopener">UCSB IPS Grant</a></td>
<td>Graduate/Postdoc</td>
<td>Funding</td>
<td>Grant for external professional development in tech (UCSB only). Up to $1200 award.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.ams.org/grants-awards/travel-grants/grad-students/emp-student-JMM" target="_blank" rel="noopener">AMS Travel Grants</a></td>
<td>Graduate</td>
<td>Funding</td>
<td>Travel grants for graduate students to attend the Joint Mathematics Meetings (JMM) and AMS sectional meetings.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://bd.csep.ucsb.edu/" target="_blank" rel="noopener">UCSB Bridge to the Doctorate Program</a></td>
<td>Undergrad</td>
<td>Funding</td>
<td>Funding and mentorship program for STEM undergraduates transitioning to graduate school (UCSB only).</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://afieldguidetomath.wordpress.com/2025/01/27/updated-mac-fund-the-mathandcobb-fund-for-mathematical-activities/" target="_blank" rel="noopener">The MaC Fund</a></td>
<td>General</td>
<td>Funding</td>
<td>Small fun grants for math activities by Professor Álvaro Lozano-Robledo. Awards fluxuate around $200.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.graddiv.ucsb.edu/our-services/central-campus-fellowships" target="_blank" rel="noopener">UCSB Central Campus Fellowships</a></td>
<td>Graduate</td>
<td>Funding</td>
<td>Fellowships for continuing UCSB graduate students (UCSB only).</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.ams.org/opportunities?utm_source=Informz&amp;utm_medium=email&amp;utm_campaign=Informz%20Mailing&amp;_zs=uxK3S1&amp;_zl=HR1r7" target="_blank" rel="noopener">AMS Mathematics Opportunities Listing</a></td>
<td>General</td>
<td>Various</td>
<td>Listing of various math opportunities by the AMS.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.simonsfoundation.org/grant/simons-dissertation-fellowship-in-mathematics/" target="_blank" rel="noopener">Simons Dissertation Fellowship</a></td>
<td>Graduate</td>
<td>Funding</td>
<td>Fellowship for PhD students in their fourth and fith years. Must apply in the third year. $38,000 award.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.gemfellowship.org/gem-fellowship-program/#a" target="_blank" rel="noopener">GEM Fellowship Program</a></td>
<td>Undergrad</td>
<td>Funding</td>
<td>Graduate School Fellowship paired with Summer internships. 5 year award.</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://www.ucsbdrp.org/" target="_blank" rel="noopener">UCSB Directed Reading Program</a></td>
<td>Undergrad</td>
<td>Mentorship/Research</td>
<td>Undergraduate research program guided by graduate student mentors during the academic year (UCSB only).</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://drive.google.com/drive/folders/0B1v4A0GzFvfHTnZRaFVDaks5R1E?resourcekey=0-3Fz_YPJ6nupoks3vq63X_w" target="_blank" rel="noopener">UCSB Math Qual Repository</a></td>
<td>Graduate</td>
<td>Math</td>
<td>Repository of previous years math qualification exams (UCSB only).</td>
</tr>
<tr>
<td class='title'><a class="resource-link" href="https://drive.google.com/drive/folders/1JJx8CO8xx3tf6zUTlJOTkuVTpQo6U0Qd" target="_blank" rel="noopener">UCSB Math Qual Solutions Repository</a></td>
<td>Graduate</td>
<td>Math</td>
<td>Repository of solutions for previous years math qualification exams (UCSB only).</td>
</tr>
</tbody>
</table>
</div>

<script src="https://code.jquery.com/jquery-3.6.0.min.js"></script>
<script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
<script>
document.addEventListener("DOMContentLoaded", function(){
  $('#resources-table').DataTable({
    responsive: true,
    pageLength: 10,
    order: [[0,'asc'], [1, 'asc'], [2, 'asc']],
    orderMulti: true,
    columnDefs: [
      { targets: [], orderable: true } // you can tweak to disable specific columns if desired
    ],
    language: { search: "Filter:" }
  });
});
</script>
